In [1]:
from benchmark import impl
from benchmark import alphaxutils
from decimal import Decimal
import pandas as pd

Order = alphaxutils.Order
tradeBySourceAmount = alphaxutils.tradeBySourceAmount
tradeByTargetAmount = alphaxutils.tradeByTargetAmount
AlphaRouter = alphaxutils.AlphaRouter
CarbonOrderUI = alphaxutils.CarbonOrderUI
assertAlmostEqual = alphaxutils.assertAlmostEqual
get_geoprice = AlphaRouter.get_geoprice

In [2]:
threshold_orders = 6
inputAmount = Decimal('100000') #63205
support_partial = True

order_params = {
    'liquidity':Decimal('5'),
    'highestRate':Decimal('0.0005'), # 2000
    'lowestRate':Decimal('0.0004'),  # 2500
    'marginalRate':Decimal('0.0005'),
    }
order_params2 = {
    'liquidity':Decimal('4'),
    'highestRate':Decimal('0.0006'), # 1666
    'lowestRate':Decimal('0.0005'),  # 2000
    'marginalRate':Decimal('0.0006'),
    }

In [3]:
1/0.0006

1666.6666666666667

In [4]:
((0.0005 * 0.0006)**0.5)

0.0005477225575051661

In [5]:
((0.0005 * 0.0006)**0.5) * 4

0.0021908902300206644

In [6]:
1/((0.0005 * 0.0006)**0.5)

1825.7418583505537

In [7]:
1/((0.0005 * 0.0006)**0.5) * 4

7302.967433402215

In [8]:
orders = []
for i in range(5):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]

indexes = list(range(len(orders)))   

In [9]:
i = 1
print(orders[i].B)
print(orders[i].A)

0.022360679774997896964091736687312762354406183596115
0.002134217652833884017881104059746151565253291210452


In [10]:
# dy_f = lambda p: sum(o.dyfromp_f(p) for o in orders)
# dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in orders)
# p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
# rl1 = [o.dyfromp_f(p_goal) for o in orders]
# rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in orders]

# actions = list(enumerate((zip(rl1, rl2))))
# # assertAlmostEqual(inputAmount, sum(rl1), Decimal('1E-8'))
# print('total_input',sum(rl1))
# print('total_output', sum(rl2))
# print('effective_price', sum(rl2) / sum(rl1))
# # return(actions)
# actions

In [11]:
get_geoprice(1, orders)

Decimal('0.00054772255750516611345696978280080213395274469499798')

# Match by Target

In [12]:
# inputAmount=Decimal('2000')
# tradeBySourceAmount(x=inputAmount, order=orders[1])

In [13]:
def matchByTarget_selector(inputAmount, orders):
    hypothetical_output_amts = {i: tradeBySourceAmount(x=inputAmount, order=orders[i])[1] for i in indexes}   # WTF not sure why this is bySource but it works - for input USDC as target, orders defined in ETH per USDC
    # max_output_amt = {i: tradeByTargetAmount(x=orders[i].y, order=orders[i])[1] for i in indexes}  
    associated_liquidity = [orders[i].y for i in hypothetical_output_amts.keys()]

    amounts = []
    effective_prices = []
    available_value = []
    for k, v in hypothetical_output_amts.items():
        if v > orders[k].y:
            # print('1')
            price = get_geoprice(k, orders)
            amount = orders[k].y
            amounts += [amount]
            effective_prices += [price]
            available_value += [amount / price]
        else:
            # print('2')
            amounts += [v]
            price = v / inputAmount
            effective_prices += [price]
            available_value += [v / price]

    results = pd.DataFrame(
        [
            hypothetical_output_amts.keys(),
            hypothetical_output_amts.values(),
            associated_liquidity,
            amounts,
            effective_prices,
            available_value,
        ],
        index=[
            "indexes",
            "hypothetical_output_amts",
            "associated_liquidity",
            "amount",
            "effective_prices",
            "available_value",
        ],
    )
    results = results.T.copy()
    results.sort_values(
        by=["hypothetical_output_amts", 'indexes'], ascending=[False, True], inplace=True
    )

    results.fillna(0, inplace=True)
    results.reset_index(inplace=True, drop=True)
    # results

    if (not support_partial) & (results.available_value.sum() < abs(inputAmount)):
        print('Insufficient Liquidity')
        return(None)
    else:
        passed_indexes = AlphaRouter.gen_one_order_selector(results.available_value, abs(inputAmount), threshold_orders)
        top_n_threshold_orders = [results.indexes[i] for i in passed_indexes]
    order_subset = [orders[i] for i in top_n_threshold_orders]
    total_subset_liquidity = results[results.indexes.isin(top_n_threshold_orders)].available_value.sum()
    return(order_subset, total_subset_liquidity, top_n_threshold_orders)

## Partial Match Logic

In [14]:
order_subset, total_subset_liquidity, top_n_threshold_orders = matchByTarget_selector(inputAmount, orders)

## Exact Matching with Marginal Price Router

In [15]:
def mpr_matchByTarget(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial):
    
    print('inputAmount', inputAmount)
    print('total_subset_liquidity', total_subset_liquidity)
    if inputAmount>= total_subset_liquidity:
        if support_partial:
            print(f'** Partial Match ({total_subset_liquidity/inputAmount*100:0.5f}%) **')
            inputAmount = total_subset_liquidity
            rl1 = [o.y for o in order_subset]
            rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
        else:
            print('Insufficient Liquidity with threshold orders')
            return(None)
    else:
        dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
        dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
        p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
        rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
        rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

    actions = {top_n_threshold_orders[i]:{"input":rl2[i],"output":rl1[i]} for i in range(len(top_n_threshold_orders))}
    assertAlmostEqual(inputAmount, sum(rl2), Decimal('1E-8'))
    print('total_input',sum(rl2))
    print('total_output', sum(rl1))
    print('effective_price', sum(rl1) / sum(rl2))
    print('1/effective_price', sum(rl2) / sum(rl1))
    return(actions)

mpr_matchByTarget(inputAmount, order_subset, total_subset_liquidity, top_n_threshold_orders, support_partial)

inputAmount 100000
total_subset_liquidity 63204.666870896957256322272155625934338718721590263
** Partial Match (63.20467%) **
total_input 63204.666870896957256322272155625934338718721590268
total_output 29
effective_price 0.00045882687838915354292782070335774230457929040542126
1/effective_price 2179.4712714102399053904231777802046323696110893196


{9: {'input': Decimal('7302.9674334022148460929304373440284527032625999732'),
  'output': Decimal('4')},
 0: {'input': Decimal('11180.339887498948482045868343656381177203091798059'),
  'output': Decimal('5')},
 2: {'input': Decimal('11180.339887498948482045868343656381177203091798059'),
  'output': Decimal('5')},
 4: {'input': Decimal('11180.339887498948482045868343656381177203091798059'),
  'output': Decimal('5')},
 6: {'input': Decimal('11180.339887498948482045868343656381177203091798059'),
  'output': Decimal('5')},
 8: {'input': Decimal('11180.339887498948482045868343656381177203091798059'),
  'output': Decimal('5')}}

# Other

In [16]:
# ## OG
# def mpr_match(inputAmount, order_subset):
#     total_subset_liquidity = sum(o.y for o in order_subset)
#     print('inputAmount', inputAmount)
#     print('total_subset_liquidity', total_subset_liquidity)
#     if inputAmount>= total_subset_liquidity:
#         print(f'Partial Match - using {total_subset_liquidity}')
#         inputAmount = total_subset_liquidity
#         rl1 = [o.y for o in order_subset]
#         rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
#     else:
#         dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
#         dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
#         p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
#         rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
#         rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

#     actions = {top_n_threshold_orders[i]:{"input":rl1[i],"output":rl2[i]} for i in range(len(top_n_threshold_orders))}
#     assertAlmostEqual(inputAmount, sum(rl1), Decimal('1E-8'))
#     print('total_input',sum(rl1))
#     print('total_output', sum(rl2))
#     print('effective_price', sum(rl2) / sum(rl1))
#     return(actions)

In [17]:
# dx_f(Decimal('0.0004'))

In [18]:
# dx_f(Decimal('0.0005'))

In [19]:
# dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
# dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
# p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
# rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
# rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]
# rl2

In [20]:
# 1/p_goal

In [21]:
# rl1

In [22]:
# inputAmount=Decimal('3')

In [23]:
# dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
# dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
# p_goal = CarbonOrderUI.goalseek(lambda p: dy_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
# rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
# rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]  # dxfromdy_f
# rl1

In [24]:
# 1/p_goal

In [25]:
# rl2

In [26]:
# order_subset[0].dxfromdy_f(1)

In [27]:
# 1/Decimal('0.00058693063937629152836424244570020053348818617374949')


In [28]:
# import matplotlib.pyplot as plt
# import numpy as np

In [29]:
# rl2

In [30]:
# mpr_match(inputAmount, order_subset)